In [2]:
import pandas as pd
from notebooks.train_test_divide import extract_spammers_scrapers, extract_normal, split_train_test, label_data
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.ensemble import IsolationForest
from sklearn import metrics
import seaborn as sns
import numpy as np
import os
import matplotlib.pyplot as plt
import math

path = os.path.dirname(os.path.abspath("__file__"))
dir = path.replace('notebooks', 'api_files')
os.chdir(dir)
from api_files.ml_library import read_csv_file, clean_reqlogs, extract_users, split_user_df, calc_avg_timediff, avg_tokens_5mins, longest_consec, get_variance_score, sequence_time_length

In [149]:
true_pos = []
true_neg = []
false_pos = []
false_neg = []

for i in range(100):
    #Creating train data
    path = os.path.dirname(os.path.abspath("__file__"))
    dir = path.replace('notebooks', 'csv_files')
    os.chdir(dir)
    
    #Train
    data_calc = pd.read_csv('csv_files/calculations80.csv')
    df_spam = extract_spammers_scrapers(data_calc)
    split_train_test(extract_normal(data_calc, df_spam), df_spam)
    df_anomaly_train = pd.read_csv('csv_files/train.csv')

    #Fit model
    anomaly_inputs = ['request_freq', 'avg_tokens', 'longest_consec', 'var_score', 'sequence_time']
    #anomaly_inputs = ['var_score', 'longest_consec']
    model_if = IsolationForest(contamination=0.002, random_state=42)
    model_if.fit(df_anomaly_train[anomaly_inputs].values)   

    #Create test data
    data_calc = pd.read_csv('csv_files/calculations80.csv')
    df_spam = extract_spammers_scrapers(data_calc)
    split_train_test(extract_normal(data_calc, df_spam), df_spam)   
    df_anomaly_test = pd.read_csv('csv_files/test.csv')


    #Code for checking how many spammers and scarpers that are present in test dataset
    spam = extract_spammers_scrapers(df_anomaly_test)
    normal = extract_normal(df_anomaly_test, spam)
    spam_true = spam.shape[0]
    normal_true = normal.shape[0]


    #Predict test data
    df_anomaly_test['anomaly_score'] = model_if.decision_function(df_anomaly_test[anomaly_inputs].values)
    df_anomaly_test['anomaly'] = model_if.predict(df_anomaly_test[anomaly_inputs].values)
    df_anomaly_score = df_anomaly_test.loc[:,['anomaly_score', 'anomaly']]
    df_anomaly_score['user'] = df_anomaly_test['user']
    df_a = df_anomaly_score.loc[df_anomaly_score['anomaly_score'] < 0] 


    spam = extract_spammers_scrapers(df_a)
    tp = spam.shape[0]
    fp = extract_normal(df_a, spam).shape[0]
    tn = normal_true - fp
    fn = spam_true - tp

    true_pos.append(tp)
    true_neg.append(tn)
    false_pos.append(fp)
    false_neg.append(fn)

In [150]:
#Values taken from average
cf_matrix = np.array([[math.floor(np.average(true_pos)), math.floor(np.average(false_neg))],[math.floor(np.average(false_pos)), math.floor(np.average(true_neg))]])
cf_matrix

array([[1674,    0],
       [   0,  424]])